# Getting Started with RAiDER

**Author**: Jeremy Maurer, David Bekaert, Simran Sangha, Yang Lei - Jet Propulsion Laboratory, California Institute of Technology

This notebook provides an overview of how to get started using the RAiDER package for estimating tropospheric RADAR delays, and other functionality included in the **raiderDelay.py** program. We give an example of how to download and process delays using ERA-5 and HRRR weather models for the Los Angeles region. 

In this notebook, we will demonstrate how to:
- Download and install the RAiDER package
- Run RAiDER to generate a grid of delays over the Los Angeles region
- Compare tropospheric delays from the weather model to that obtained from a GNSS station
    
<div class="alert alert-warning">
The initial setup (<b>Prep A</b> section) should be run at the start of the notebook. The overview sections do not need to be run in order. 
</div>

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    
- GDAL uses "HDF5" driver instead of "netCDF/Network Common Data Format." Verify GDAL version >= 3.
- RAiDER needs to be installed to run this notebook
</div>


<div class="alert alert-info">
    <b>Terminology:</b>
    
- *Acquisition*: A single image acquired by a satellite at a particular time
- *Interferogram*: An unwrapped image containing the surface displacement accumulated between two acquisitions.
- *Weather model*: A reanalysis weather product defining temperature, pressure, and humidity on a regular grid in some coordinate system (e.g., at regular lat/lon intervals in the standard EPSG:4326 reference frame).
- *Delay*: The apparent displacement in an interferogram that occurs solely due to changes in weather conditions between two acquisitions.
    </div>
    

## Prep A. Initial setup of the notebook

Below we set up the directory structure for this notebook exercise. In addition, we load the required modules into our python environment using the **`import`** command. We also explicitly enable exceptions for GDAL as this allows us to capture GDAL errors.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Defining the home and data directories at the processing location
work_dir = os.path.abspath(os.getcwd())
tutorial_home_dir = os.path.abspath(os.getcwd())
print("Work directory: ", work_dir)
print("Tutorial directory: ", tutorial_home_dir)

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Verifying if ARIA-tools is installed correctly
try:
    from RAiDER.delay import tropo_delay
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')
        
os.chdir(work_dir)

ModuleNotFoundError: No module named 'matplotlib'

Below we define a plotting function that will be used throughout the notebook for plotting GDAL compatible datasets on a map.

In [2]:
def plot_layer(path_layer, lay_type=None, cmap=None, n_bands=None, **kwargs):
    """ 
        path_layers is a string to the GDAL compatible dataset to be plotted
    """
    
    if not lay_type: 
        lay_type = os.path.dirname(path_layer)
    title = [os.path.basename(lay_type)]
    
    ## get the lon lat bounds
    ds       = gdal.Open(path_layer, gdal.GA_ReadOnly)
    trans    = ds.GetGeoTransform()
    extent   = [trans[0], trans[0] + ds.RasterXSize * trans[1], trans[3] + ds.RasterYSize*trans[5], trans[3]]
    
    ## loading the data
    if not n_bands:
        n_bands  = ds.RasterCount
    lst_arrs = []
    
    for band in range(n_bands):
        raster = ds.GetRasterBand(band+1)
        arr    = raster.ReadAsArray()
        try:
            NoData = raster.GetNoDataValue()
            arr = np.ma.masked_where((arr>1e20) |(arr==NoData),arr )
        except:
            print('Could not find a no-data value...')
            arr = np.ma.masked_where(arr>1e20,arr)
        
        lst_arrs.append(arr)

    ds = None
    if n_bands < 4:
        nrows = 1; ncols = n_bands
    else:
        raise Exception('Number of bands currently unsupported')
        
    
    ## initializing a figure
    fig, axes = plt.subplots(figsize=(12,9), ncols=ncols, nrows=nrows, sharex='col', sharey='row')
    axes = axes if isinstance(axes, np.ndarray) else np.array(axes)
    axe  = axes.ravel() 
    cmap = plt.cm.Greys_r
    cmap.set_under('black')
    
    ## definging the plotting options for differnt layer types
    # Amplitude:
    if lay_type.endswith('amplitude'): 
        # will fix the maximum amplitude bound
        vmin=None
        vmax = 2000 
    # Coherence:
    elif lay_type.endswith('coherence'): 
        # has fixed range between 0-1
        vmin=0
        vmax = 1
    # Incidence angle:
    elif lay_type.endswith('incidenceAngle'): 
        vmin=None
        vmax=None
    # water
    elif lay_type.startswith('water'):
        # no bounds needed will be a 0/1 mask
        vmin=0
        vmax=1
        cmap='Greys'
    # deformation or unwrapped phase
    elif lay_type.startswith('defo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
    elif lay_type.startswith('terr') or lay_type.startswith('topo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.terrain
    elif lay_type == 'ENU':
        vmin=None
        vmax=None
        title = ['East', 'North', 'Up']
        fig.subplots_adjust(wspace=0.5)

    else:
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
        
    # plotting the data    
    for i, ax in enumerate(axe):
        im   = ax.imshow(lst_arrs[i], cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
        divider = make_axes_locatable(ax)
        cax     = divider.append_axes('right', size='5%', pad=0.25)
        if lay_type == 'ENU':
            fig.colorbar(im, cax=cax, format=FuncFormatter(lambda x, y: '{:.3f}'.format(x)))
        elif lay_type.startswith('water'):
            fig.colorbar(im, cax=cax, ticks=[vmin, vmax])
        else:
            fig.colorbar(im, cax=cax)

        ax.set_title(title[i], fontsize=15)
        ax.grid(False)

    axe[0].set_ylabel('latitude', labelpad=15, fontsize=15)
# Setup for accessing weather model data

## For accessing ECMWF-type weather model data (e.g. ECMWF, ERA5, ERA5-T)

1. Create an account on the Copernicus servers [here](https://cds.climate.copernicus.eu/user)

2. Confirm your email, etc. 

3. Install the public API key and client as instructed [here](https://cds.climate.copernicus.eu/api-how-to): 

   a. Copy the URL and API key from the webpage into a file in your home directory name ~/.cdsapirc 
      
         url: https://cds.climate.copernicus.eu/api/v2
         key: <KEY>
      
      _Note the `<KEY>` represents the API key obtained upon the registration of CDS API, and should be replaced with the user's own information._
      
   b. Install the CDS API using pip: 
   
         pip install cdsapi
   
   ___Note: this step has been included in the conda install of RAiDER, thus can be omitted if one uses the recommended conda install of RAiDER___
   
4. You must accept the [license](https://cds.climate.copernicus.eu/cdsapp/#!/terms/licence-to-use-copernicus-products) for each product you wish to download.

## For accessing MERRA2-type weather model data with the use of OpenDAP  (e.g. MERRA-2, GMAO)

1. Create an account on the NASA's Earthdata website [here](https://urs.earthdata.nasa.gov)

2. Confirm your email, etc. 

3. Copy the login username and password to a file in your home directory name ~/.netrc 
         
         machine urs.earthdata.nasa.gov
                 login <USERNAME>
                 password <PASSWORD>
                 
   _Note the `<USERNAME>` and `<PASSWORD>` represent the actual username and password, and should be replaced with the user's own information correspondingly._
   
4. Add the application `NASA GESDISC DATA ARCHIVE` by clicking on the `Applications->Authorized Apps` on the menu after logging into your Earthdata profile, and then scrolling down to the application `NASA GESDISC DATA ARCHIVE` to approve it. _This seems not required for GMAO for now, but recommended to do so for all OpenDAP-based weather models._

5. Install the OpenDAP using pip: 

         pip install pydap==3.2.1
      
   ___Note: this step has been included in the conda install of RAiDER, thus can be omitted if one uses the recommended conda install of RAiDER___
   
   ___Note: PyDAP v3.2.1 is required for now (thus specified in the above pip install command) because the latest v3.2.2 (as of now) has a known [bug](https://colab.research.google.com/drive/1f_ss1Oa3VzgAOd_p8sgekdnLVE5NW6s5) in accessing and slicing the GMAO data. This bug is expected to be fixed in newer versions of PyDAP.___    axe[int(np.floor(n_bands/2))].set_xlabel('longitude', labelpad=15, fontsize=15)

# Setup for accessing weather model data

## For accessing ECMWF-type weather model data (e.g. ECMWF, ERA5, ERA5-T)

1. Create an account on the Copernicus servers [here](https://cds.climate.copernicus.eu/user)

2. Confirm your email, etc. 

3. Install the public API key and client as instructed [here](https://cds.climate.copernicus.eu/api-how-to): 

   a. Copy the URL and API key from the webpage into a file in your home directory name ~/.cdsapirc 
      
         url: https://cds.climate.copernicus.eu/api/v2
         key: <KEY>
      
      _Note the `<KEY>` represents the API key obtained upon the registration of CDS API, and should be replaced with the user's own information._
      
   b. Install the CDS API using pip: 
   
         pip install cdsapi
   
   ___Note: this step has been included in the conda install of RAiDER, thus can be omitted if one uses the recommended conda install of RAiDER___
   
4. You must accept the [license](https://cds.climate.copernicus.eu/cdsapp/#!/terms/licence-to-use-copernicus-products) for each product you wish to download.

## For accessing MERRA2-type weather model data with the use of OpenDAP  (e.g. MERRA-2, GMAO)

1. Create an account on the NASA's Earthdata website [here](https://urs.earthdata.nasa.gov)

2. Confirm your email, etc. 

3. Copy the login username and password to a file in your home directory name ~/.netrc 
         
         machine urs.earthdata.nasa.gov
                 login <USERNAME>
                 password <PASSWORD>
                 
   _Note the `<USERNAME>` and `<PASSWORD>` represent the actual username and password, and should be replaced with the user's own information correspondingly._
   
4. Add the application `NASA GESDISC DATA ARCHIVE` by clicking on the `Applications->Authorized Apps` on the menu after logging into your Earthdata profile, and then scrolling down to the application `NASA GESDISC DATA ARCHIVE` to approve it. _This seems not required for GMAO for now, but recommended to do so for all OpenDAP-based weather models._

5. Install the OpenDAP using pip: 

         pip install pydap==3.2.1
      
   ___Note: this step has been included in the conda install of RAiDER, thus can be omitted if one uses the recommended conda install of RAiDER___
   
   __Note: PyDAP v3.2.1 is required for now (thus specified in the above pip install command) because the latest v3.2.2 (as of now) has a known [bug](https://colab.research.google.com/drive/1f_ss1Oa3VzgAOd_p8sgekdnLVE5NW6s5) in accessing and slicing the GMAO data. This bug is expected to be fixed in newer versions of PyDAP.__



## For accessing HRRR products 
1. High-resolution rapid refresh (HRRR) weather model data products are generated by [NOAA]() but not archived beyond three days. However a public HRRR archive is available at the University of Utah [archive](home.chpc.utah.edu/~u0553130/Brian_Blaylock/hrrr_FAQ.html). This archive does not require a license agreement


## Download options

ECMWF, ERA-5, ERA-5T, GMAO, MERRA-2 and HRRR weather model products can be downloaded automatically in RAiDER. Other weather models can be specified manually using the WRF or -wmnetcdf options and described in the docs. 

<div class="alert alert-warning">
<b>Potential download failure:</b> 
ERA-5/ERA-I products require access to the ESA Copernicus servers. GMAO and MERRA-2 products require access to the NASA Earthdata servers. If you are unable to download products, ensure that you have registered and have downloaded the public API key, and accepted/added the license/application for type of product you wish to download as detailed above. 
</div>

## Overview of the raiderDelay.py program
<a id='overview'></a>

The **`raiderDelay.py`** program allows for easy downloading and processing of tropospheric weather delays for InSAR correction. Running **`raiderDelay.py`** with the **`-h`** option will show the parameter options. 

Let us explore these options:

In [3]:
!raiderDelay.py -h

/bin/sh: raiderDelay.py: command not found


### 1. Date or date list (--date)

This argument is required unless the weather model files are directly specified. The date passed can be either: 
1) a single date, specified in psuedo-ISO 8601 format: 20180101, 2018-01-01, etc. 
2) a comma-deliminated pair of dates, in which case all the dates between the pair listed will be downloaded and processed. E.g., '20180101,20190101'

### 2. Time of day (--time)

This argument is also required unless files are explicitly passed. Specify the time of day to download and process in psuedo-ISO 8601 format: 
1) T020000
2) T02:00:00.000
3) T0200
etc. 

### 3. Line-of-sight vector file (-l/--lineofsight)

This option can be used to specify a line-of-sight file such as those generated as outputs from the ISCE software package for InSAR (github.com/isce-framework/isce2). This should be a two-band GDAL-readable raster file containing the look vector components, with the incidence angle in band 1 and the heading in band 2.

### 4. Statevector file (-s/--statevectors)

This should be an ISCE-derived XML file or a shelve file containing state vectors specifying the orbit of the sensor. 

### Options for Specifying Area/points to process

### 5. Lat/Lon raster files area option (-/--area)

A two-argument option giving the latitude and longitude raster files, such as would be output from the ISCE processor. 

### 6. Bounding box area option (-modelbb/--modelBBOX

This option takes four floats that specify a bounding box (given as North West South East) to be processed. 

### 7.  Station file area option (--station_file)

Instead of a lat/lon file pair or bounding box, a list of lat/lon locations (i.e., stations) can be specified. This should be a .csv file contiaining at least the columns Lat and Lon. 

### DEM (-d/--dem)

The DEM over the area of interest can be specified explicitly, or it will be downloaded on-the-fly. RAiDER will check the default location for a DEM (./geom), so if you download the DEM once it will not be downloaded again. 

### Height levels (--heightlvs)

This option specifies a list of heights, for which the delay will be calculated. The result is a full 3-D cube of delays at the specified lat/lon grid and height levels. 

### Specified weather model files (--wmnetcdf)

Allows for directly passing a list of netcdf files (the appropriate reader must also be specified) to RAiDER. 

## Other input options

### Weather model file directory location (-w/--weatherModelFileLocation)

Specifies a directory to store the original weather model files. If not specified, the default location is ./weather_files.

### Reference integration height (-z/--zref)

This option allows the user to specify the integration height when computing the total delay. The default is 15 km. 

### Output file format (--outformat)

This option is only used if the inputs are rasters or a bounding box, otherwise the output format is fixed (.csv file for station list, HDF5 file for height levels specified). Must be GDAL-compatible. 

### Output file directory (--out)

This specifies the location of the output files. If not specified the default is ./results

### Parallel Computation flag (-p, --no_parallel)

In [4]:
If specified, do not run in parallel. Off by default. 

SyntaxError: invalid syntax (<ipython-input-4-d220c037c411>, line 1)

### Download the weather model only (--download_only)

If specified, will only download the weather model and do nothing else. 

### Run in verbose mode (-v/--verbose)

Runs the code in verbose mode. Will save the weather model to a pickle file for inspection and create debugging plots.

## WRF-specific options

### WRF model files (--wrfmodelfiles)

A two-argument list of files (out plev) specfied for the WRF model

# RAiDER Readers

Weather model readers provide the link between the raw weather model data (e.g. available from ECMWF, ERA-5, ERA-5T, GMAO, MERRA-2, HRRR), and the absolute delay calculation. Readers can be added by users to account for other models and custom formats. Here we provide an overview of the WeatherModel class object and requirements for writing one's own reader function. 

## The WeatherModel class

### Functions to be overloaded:
\_fetch: 
- Called by WeatherModel.fetch method
- downloads or loads data from the source files

load_weather: 
- Called by the WeatherModel.load method
- loads data from the raw weather model files and parses it into the WeatherModel format (see below)

### Defining the Reader

The readers are defined with respect to the base WeatherModel class. At minimum, the \_\_init__, \_fetch, Name, and load_weather methods are required. 

A number of important parameters can be defined within the \_\_init__ method, if they are not specified, the defaults listed in the base WeatherModel class will be used.

The following file should be created and saved as "abcd.py" (for the custom weather model named "ABCD") under the directory of "tools/RAiDER/models". 

In [ ]:
from RAiDER.models.weatherModel import WeatherModel

class customModelReader(WeatherModel):
    def __init__(self):
        WeatherModel.__init__(self)
        self._humidityType = 'q'  # can be "q" (specific humidity) or "rh" (relative humidity)
        self._model_level_type = 'pl' # Default, pressure levels are "pl", and model levels are "ml"
        self._classname = 'abcd'  # name of the custom weather model
        self._dataset = 'abcd'  # same name as above

        # Tuple of min/max years where data is available. 
        #  valid range of the dataset. Users need to specify the start date and end date (can be "present")
        self._valid_range = (datetime.datetime(2016,7,15),"Present")  
        #  Availability lag time. Can be specified in hours "hours=3" or in days "days=3"
        self._lag_time = datetime.timedelta(hours=3) 

        # model constants (these three constants are borrowed from ECMWF model and currently
        # set to be default for all other models, which may need to be double checked.)
        self._k1 = 0.776  # [K/Pa]
        self._k2 = 0.233 # [K/Pa]
        self._k3 = 3.75e3 # [K^2/Pa]

        # horizontal grid spacing
        self._lat_res = 3./111  #  grid spacing in latitude
        self._lon_res = 3./111  #  grid spacing in longitude
        self._x_res = 3.        #  x-direction grid spacing in the native weather model projection 
                                #  (if the projection is in lat/lon, it is the same as "self._lon_res")
        self._y_res = 3.        #  y-direction grid spacing in the weather model native projection
                                #  (if the projection is in lat/lon, it is the same as "self._lat_res")

        self._Name = 'ABCD' #  name of the custom weather model (better to be capitalized)

        # Projections in RAiDER are defined using pyproj (python wrapper around Proj)
        # If the projection is defined with EPSG code, one can use "self._proj = CRS.from_epsg(4326)" 
        # to replace the following lines to get "self._proj".
        # Below we show the example of HRRR model with the parameters of its Lambert Conformal Conic projection
        lon0 = 262.5
        lat0 = 38.5
        lat1 = 38.5
        lat2 = 38.5
        x0 = 0
        y0 = 0
        earth_radius = 6371229
        p1 = CRS('+proj=lcc +lat_1={lat1} +lat_2={lat2} +lat_0={lat0} +lon_0={lon0} +x_0={x0} +y_0={y0} +a={a} +b={a} +units=m +no_defs'.format(lat1=lat1, lat2=lat2, lat0=lat0, lon0=lon0, x0=x0, y0=y0, a=earth_radius))
        self._proj = p1
       
    def _fetch(self, lats, lons, time, out, Nextra=2):
        '''
        Fetch weather model data from the custom weather model "ABCD"
        Inputs (no need to change in the custom weather model reader): 
        lats - latitude 
        lons - longitude 
        time - datatime object (year,month,day,hour,minute,second)
        out - name of downloaded dataset file from the custom weather model server
        Nextra - buffer of latitude/longitude for determining the bounding box 
        '''
        
        # bounding box plus a buffer using the helper function from the WeatherModel base class
        # This part can be kept without modification.
        lat_min, lat_max, lon_min, lon_max = self._get_ll_bounds(lats, lons, Nextra)
        self._bounds = (lat_min, lat_max, lon_min, lon_max)
        
        # Auxilliary function: 
        # download dataset of the custom weather model "ABCD" from a server and then save it to a file named out.
        # This function needs to be writen by the users. For download from the weather model server, the weather model 
        # name, time and bounding box may be needed to retrieve the dataset; for cases where no file is actually 
        # downloaded, e.g. the GMAO and MERRA-2 models using OpenDAP, this function can be omitted leaving the data 
        # retrieval to the following "load_weather" function.
        self._files = self._download_abcd_file(out, 'abcd', time, self._bounds)
    
    def load_weather(self, filename):
        '''
        Load weather model variables from the downloaded file named filename
        Inputs: 
        filename - filename of the downloaded weather model file 
        '''
        
        # Auxilliary function:
        # read individual variables (in 3-D cube format with exactly the same dimension) from downloaded file
        # This function needs to be writen by the users. For downloaded file from the weather model server, 
        # this function extracts the individual variables from the saved file named filename; 
        # for cases where no file is actually downloaded, e.g. the GMAO and MERRA-2 models using OpenDAP, 
        # this function retrieves the individual variables directly from the weblink of the weather model.
        lats, lons, xs, ys, t, q, p, hgt = self._makeDataCubes(filename)

        ########### extra steps that may be needed to calculate topographic height and pressure level if not provided
        ########### directly by the weather model through the above auxilliary function "self._makeDataCubes"
        
        # if surface pressure (in logarithm) is provided as "p" along with the surface geopotential "z" (needs to be
        # added to the auxilliary function "self._makeDataCubes"), one can use the following line to convert to 
        # geopotential, pressure level and geopotential height; otherwise commented out
        z, p, hgt = self._calculategeoh(z, p)
        
        # if the geopotential is provided as "z" (needs to be added to the auxilliary function "self._makeDataCubes"),
        # one can use the following line to convert to geopotential height; otherwise, commented out
        hgt = z / self._g0
        
        # if geopotential height is provided/calculated as "hgt", one can use the following line to convert to 
        # topographic height, which is then automatically assigned to "self._zs"; otherwise commented out
        self._get_heights(lats, hgt)
        
        # if topographic height is provided as "hgt", use the following line directly; otherwise commented out
        self._zs = hgt
        
        ###########

        ######### output of the weather model reader for delay calculations (all in 3-D data cubes) ########
        
        # _t: temperture
        # _q: either relative or specific humidity
        # _p: must be pressure level
        # _xs: x-direction grid dimension of the native weather model coordinates (if in lat/lon, _xs = _lons)
        # _ys: y-direction grid dimension of the native weather model coordinates (if in lat/lon, _ys = _lats)
        # _zs: must be topographic height
        # _lats: latitude
        # _lons: longitude
        self._t = t
        self._q = q
        self._p = p
        self._xs = xs
        self._ys = ys
        self._lats = lats
        self._lons = lons
        
        ###########

    def _download_abcd_file(self, out, model_name, date_time, bounding_box):
        '''
        Auxilliary function:
        Download weather model data from a server
        Inputs: 
        out - filename for saving the retrieved data file 
        model_name - name of the custom weather model 
        date_time - datatime object (year,month,day,hour,minute,second)
        bounding_box - lat/lon bounding box for the region of interest
        Output: 
        out - returned filename from input
        '''
        pass
        
    def _makeDataCubes(self, filename):
        '''
        Auxilliary function:
        Read 3-D data cubes from downloaded file or directly from weather model weblink (in which case, there is no 
        need to download and save any file; rather, the weblink needs to be hardcoded in the custom reader, e.g. GMAO)
        Input: 
        filename - filename of the downloaded weather model file from the server
        Outputs: 
        lats - latitude (3-D data cube)
        lons - longitude (3-D data cube)
        xs - x-direction grid dimension of the native weather model coordinates (3-D data cube; if in lat/lon, _xs = _lons)
        ys - y-direction grid dimension of the native weather model coordinates (3-D data cube; if in lat/lon, _ys = _lats)
        t - temperature (3-D data cube)
        q - humidity (3-D data cube; could be relative humidity or specific humidity)
        p - pressure level (3-D data cube; could be pressure level (preferred) or surface pressure)
        hgt - height (3-D data cube; could be geopotential height or topographic height (preferred))
        '''
        pass
    

### Required data format

The result of running the load_weather method should be a Python object with attributes consistent with the WeatherModel class and the RAiDER convention. The required variables are: 
- _lats, _lons
- _xs, _ys, _zs
- _p, _t
- _rh OR _q, matching the corresponding _humidityType

Each of these variables should be a 3-D cube, all of the same shape. 

<div class="alert alert-warning">
The '_zs' variable should be topographic height, but the height variable passed with the weather model data is often the geopotential height, which must be converted to topographic height. The WeatherModel class has a helper function for this conversion, which can be called within the custom class as self._get_heights(lats, geo_hgt), where geo_hgt is geopotential height. 
</div>

 ### Adding the reader to the weather model list

Modify the allowed list of weather models "allowed.py" under the directory of "tools/RAiDER/models" to include the custom "ABCD" model as below. 

In [1]:
ALLOWED_MODELS = [
    'ERA5',
    'ERA5T',
    'ERAI',
    'MERRA2',
    'WRF',
    'HRRR',
    'GMAO',
    'HDF5',
    'ABCD'
]

 ### Testing the custom weather model reader

Run the three example commands from the **`raiderDelay.py`** helper message (i.e. running **`raiderDelay.py`** with the **`-h`** option will show the three example commands) with the weather model name "ERA5" replaced with the newly-added custom one, e.g. "ABCD".

### Debugging
The WeatherModel class has two built-in plots for debugging purposes, which can be called using 